# GEM Control Cookbook

This notebook aims to provide an introduction to the usage of the gem-control toolbox. The first section introduces the basic functions of a GEM Controller. Afterwards, an introduction on how to tune a controller is given. Finally, some examples are presented.

##  1. Overview

The gem-control package is a Python toolbox for the control of a [gym-electric-motor](https://upb-lea.github.io/gym-electric-motor/) (GEM) environment. The toolbox automatically creates a controller structure for a specified motor and controlled state. Furthermore, it is possible to parameterize the controller automatically.

### 1.1 Installation

Before you can start, you need to make sure taht you have gem-controll installed. You can install it easily using pip:
```
pip install gem-control
```

Alternatively, install the latest developer version directly from GitHub:
https://github.com/upb-lea/gem-control

For this notebook, we can install it by executing the following cell.

In [ ]:
!pip install -q git+https://github.com/upb-lea/gem-control@main

In [ ]:
!sudo apt-get install latexmk -qq
!sudo apt-get install texlive-latex-extra -qq

In [ ]:
### 1.2 Supported Motors

The following motors are supported to be controlled by a GEM Controller. Three options are available for the control task: Current Control (CC), Torque Control (TC) and Speed Control (SC).

DC Motors:
   - Permanently Excited DC Motor
   - Externally Excited DC Motor
   - Series DC Motor
   - Shunt DC Motor

Three-Phase Motors:
   - Permanent Magnet Synchronous Motor
   - Synchronous Reluctance Motor
   - Externally Excited Synchronous Motor
   - Squirrel Cage Induction Motor

### 1.2 Supported Motors

The following motors are supported to be controlled by a GEM Controller. Three options are available for the control task: Current Control (CC), Torque Control (TC) and Speed Control (SC).

DC Motors:
   - Permanently Excited DC Motor
   - Externally Excited DC Motor
   - Series DC Motor
   - Shunt DC Motor

Three-Phase Motors:
   - Permanent Magnet Synchronous Motor
   - Synchronous Reluctance Motor
   - Externally Excited Synchronous Motor
   - Squirrel Cage Induction Motor

### 1.3 Interface

The basic user interface consists of four main functions.
* `import gem_controller as gc`
      Import the package.
    

* `c = gc.GemControllers.make(environment, environment-id, **kwargs)`
      Returns an instantiated controller for the gem-environment. Call this function at the beginning.


* `c.reset()`
      Resets the controller. Call this function before a new episode starts.


* `c.control(state, reference)`
      Returns the action of the controller for the next step.


* `c.control_environment(environment, n_steps, **kwargs)`
      Controls the environment for n timesteps.
 

### 1.4 Basic Routine

A Basic Routine of creating a gem controller and using it, to control a gym-electric-motor environment is given in the following cell. For setting up a gym-electric-motor environment, a seperate [GEM Cookbook](https://colab.research.google.com/github/upb-lea/gym-electric-motor/blob/master//examples/environment_features/GEM_cookbook.ipynb) is available.

In [ ]:
import gym_electric_motor as gem
import gem_controllers as gc
%matplotlib notebook


# Instantiate a gym-electric-motor environment
env_id = 'Cont-CC-PMSM-v0'
env = gem.make(env_id)

# Instantiate a gem controller for the environment
c = gc.GemController.make(env, env_id)

# Control the environment
c.control_environment(env, n_steps=10001, render_env=True)


<div class="alert alert-block alert-info">  
<b>Note:</b>    
If the notebook is executed in <b>Colab</b>, the visualization cannot be used because dynamic matplotlib backends are not supported.
</div>

## 2. Customize a GEM Controller

### 2.1 Creating a GEM Controller

To create a GEM Controller the `GemControllers.make()` command is used. There are some keyword arguments to adjust the controller.

* **env**
      The GEM environment that should be controlled.

* **env_id**
      The id of the GEM environment, that includes the action space, the control task and the motor type.

* **decoupling**
      Boolean to indicate, if an EMF-Feedforward corretion stage should be used.
        

* **current_safety_margin**
      Float to define a maximum current for the operation point selection.

* **base_speed_controller**
      Selection of the basic control algorithm for the speed control stage. Available control algorithms are 'PI', 'PID', 'P', 'ThreePoint'.

* **base_current_controller**
      Selection of the basic control algorithm for the current control stage. Available control algorithms are 'PI', 'PID', 'P', 'ThreePoint'.
      
* **a**
      Tuning parameter of the symmetrical optimum, that has to be larger than 1. A small value leads to a fast response, but also to a high overshoot.

* **plot_references**
      Boolean to indicate, if the reference values of the subordinated stages should be plotted. Therefore the referenced states of the subordinated stages have to be plotted.

* **block_diagram**
      Boolean to indicate, if a block diagram of the controller should be generated.

* **save_block_diagram_as**
      Selection of the data type for saving the block diagram. Also a tuple of data types can be passed. The available data types are 'pdf' and 'tex'. A window to select the folder and the file name will be opened.

In [ ]:
from gym_electric_motor.visualization import MotorDashboard
from gym_electric_motor.reference_generators import WienerProcessReferenceGenerator

# Create a GEM environment
env_id = 'Cont-SC-PMSM-v0'
env = gem.make(env_id,
              visualization=MotorDashboard(state_plots=('omega', 'torque', 'i_sd', 'i_sq', 'u_sd', 'u_sq')),
              reference_generator=WienerProcessReferenceGenerator(reference_state='omega', sigma_range=(1e-3, 1e-2)))

# Create a GEM controller
c = gc.GemController.make(env=env,
                          env_id=env_id,
                          decoupling=False,
                          current_safety_margin=0.3,
                          base_speed_controller='PI',
                          base_current_controller='PI',
                          a=5,
                          plot_references=True,
                          block_diagram=True,
                          save_block_diagram_as=())


### 2.2 Controlling a GEM environment

To use a GEM controller for controlling a GEM environment, the function `c.control_environment()` is available. There are some arguments, that could be passed.

* **env**
      The GEM environment to be controlled. This should be the same environment, that was used for the make command of the controller

* **n_steps**
      The number of iteration steps.

* **max_episode_length**
      The maximum number of steps, before the environment and the controller are reset.
      
* **render_env**
      Boolean to indicate, if the states of the environment should be plotted.

In [ ]:
# Control the environment by the build in function
c.control_environment(env=env,
                      n_steps=30001,
                      max_episode_length=10000,
                      render_env=True)

If additional calculations should be performed during the simulation, the control loop can be programmed manually.

In [ ]:
# Reset the environment and the controller
state, reference = env.reset()
c.reset()

c.block_diagram.show()   # Show the block diagram of the controller

n_steps = 10001

cumulative_reward = 0

for _ in range(n_steps):     # Simulate the environment and controller for n steps
    env.render()      # Plot the states of the environment
    action = c.control(state, reference)     # Calculate the action
    (state, reference), reward, done, _ = env.step(action)    # Simulate one step of the environment
    
    cumulative_reward += reward
    
    if done:      # Check, if the system limits are exceeded
        state, reference = env.reset()       # Reset the environment
        c.reset()      # Reset the controller

print('Cumulative Reward:', cumulative_reward)

<div class="alert alert-block alert-info">  
<b>Note:</b>    
The state plots can be seen in the previous cell, because the same environment is used. 
</div>

## 3. Examples

### 3.1 Current control of a Series DC Motor

In [ ]:
env_id = 'Finite-CC-SeriesDc-v0'
env = gem.make(env_id,
               visualization=MotorDashboard(state_plots=('torque', 'i'), action_plots='all'),
               reference_generator=WienerProcessReferenceGenerator(reference_state='i', sigma_range=(1e-3, 1e-2)))

c = gc.GemController.make(env, env_id)

c.control_environment(env, n_steps=10001, render_env=True)

### 3.2 Torque control of an Externally Excited DC Motor

The torque of an Externally Excited DC Motor can be adjusted by the $i_{\mathrm{a}}$ and $i_{\mathrm{e}}$ current. The current operation point is selected in order to minimize the ohmic losses.

In [ ]:
env_id = 'Cont-TC-ExtExDc-v0'
env = gem.make(env_id,
               visualization=MotorDashboard(state_plots=('torque', 'i_a', 'i_e', 'u_a', 'u_e')),
               reference_generator=WienerProcessReferenceGenerator(reference_state='torque', sigma_range=(1e-3, 1e-2)))

c = gc.GemController.make(env, env_id)

c.control_environment(env, n_steps=10001, render_env=True)

### 3.3 Torque control of an Externally Excited Synchronuous Motor

In [ ]:
env_id = 'Cont-TC-EESM-v0'
env = gem.make(env_id,
               visualization=MotorDashboard(state_plots=('torque', 'i_sd', 'i_sq', 'i_e', 'u_sd', 'u_sq', 'u_e')),
               reference_generator=WienerProcessReferenceGenerator(reference_state='torque', sigma_range=(1e-3, 1e-2)))

c = gc.GemController.make(env, env_id, a=5)

c.control_environment(env, n_steps=10001, render_env=True)

### 3.4 Torque Control of a Squirrel Cage Induction Motor

For the control of a Squirrel Cage Induction Motor (SCIM) a flux observer is necessary, to obtain an estimate value for the magnetic flux. A flux observer is implemented as a physical system wrapper in the GEM toolbox. This has to be used, when controlling an SCIM.

In [ ]:
from gym_electric_motor.physical_system_wrappers import FluxObserver
from gym_electric_motor.reference_generators import SinusoidalReferenceGenerator

env_id = 'Cont-TC-SCIM-v0'
env = gem.make(env_id,
               physical_system_wrappers=(FluxObserver(),),
               visualization=MotorDashboard(state_plots=('torque', 'i_sd', 'i_sq', 'u_sd', 'u_sq', 'psi_abs')),
               reference_generator=SinusoidalReferenceGenerator(reference_state='torque', episode_lengths=5000,))

c = gc.GemController.make(env, env_id, a=5, current_safety_margin=0.1)

c.control_environment(env, n_steps=10001, render_env=True)

### 3.5 Speed Control of a Shunt DC Motor

In [ ]:
env_id = 'Cont-SC-ShuntDc-v0'
env = gem.make(env_id,
               visualization=MotorDashboard(state_plots=('omega', 'torque', 'i_a', 'u')),
               reference_generator=WienerProcessReferenceGenerator(reference_state='omega', sigma_range=(5e-4, 8e-4)))

c = gc.GemController.make(env, env_id)

c.control_environment(env, n_steps=10001, render_env=True)